In [ ]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from pathlib import Path
import pandas as pd
import h5py
import re
import os 
import json 
import os
import pickle  # You can choose another format if preferred


In [ ]:
# File paths
folderpath = Path(r'F:\AlterG\Control\Data')
HeelPathTemplate = r'F:\AlterG\Control\Data\{}\Gait\DevicesData.mat'
header_line = 8
pattern = r"\\([^\\]+)_ik"
base_json_path = r"F:\AlterG\Control\InverseKinematics"


In [ ]:
# Dictionaries
concatenated_stride_data = {}
all_strides_dict = {'Right': {}, 'Left': {}}


In [ ]:
# Function to interpolate strides to a common length and concatenate them
def interpolate_strides(strides, target_length=100):
    interpolated_values_list = []
    x_new = np.linspace(0, 1, target_length)
    
    for stride_array in strides:
        for stride in stride_array:
            if len(stride) < 1:
                continue
            
            x_old = np.linspace(0, 1, len(stride))
            
            if len(stride) < 4:
                interp_func = interp1d(x_old, stride, kind='linear', bounds_error=False, fill_value="extrapolate")
            else:
                interp_func = interp1d(x_old, stride, kind='cubic', bounds_error=False, fill_value="extrapolate")
            
            interpolated_values = interp_func(x_new)
            interpolated_values_list.append(interpolated_values)
    
    return np.array(interpolated_values_list)


In [ ]:
# Function to plot all strides per header in a single plot
def plot_stride_list(strideList, side, header):
    """    
    Parameters:
    strideList (list): List containing the stride data.
    side (str): Side of the stride ('Right' or 'Left').
    header (str): The header to plot.

    """
    plt.figure(figsize=(12, 6))
    for stride in strideList:
        plt.plot(stride, label=f'{side} Side', alpha=0.6)
    
    plt.title(f'All Strides for {header} ({side} Side)')
    plt.xlabel('Sample Points')
    plt.ylabel('Values')
    #plt.legend()
    plt.show()
    plt.close()


In [ ]:
# Function to plot all interpolated strides per header in a single plot.
def plot_interpolated_values(interpolated_values, side, header):
    """
    Parameters:
    interpolated_values (ndarray): Array containing the interpolated stride data.
    side (str): Side of the stride ('Right' or 'Left').
    header (str): The header to plot.

    """
    plt.figure(figsize=(12, 6))
    for stride in interpolated_values:
        plt.plot(stride, alpha=0.6, label=f'{side} Side' if side not in plt.gca().get_legend_handles_labels()[1] else "")
    
    plt.title(f'Interpolated Strides for {header} ({side} Side)')
    plt.xlabel('Sample Points')
    plt.ylabel('Values')
    #plt.legend()
    plt.show()


In [ ]:
# Function to plot concatenated strides for each header and side
def plot_subject(concatenated_stride_data, side, header):
    plt.figure(figsize=(12, 6))
    for stride_list in concatenated_stride_data[header][side]:
        for stride in stride_list:
            plt.plot(stride, alpha=0.6, label=f'{side} Side' if side not in plt.gca().get_legend_handles_labels()[1] else "")
    
    plt.title(f'Individual Strides for {header} ({side} Side) for {trial}')
    plt.xlabel('% Gait Cycle')
    plt.ylabel(f'{header} (deg)')
    #plt.legend()
    plt.show()
    plt.close()


In [ ]:
# Function to plot mean and standard deviation for each header and side
def plot_mean_std(concatenated_stride_data):
    mean_strides_dict = {}
    std_strides_dict = {}

    for side in ['Right', 'Left']:
        mean_strides_dict[side] = {}
        std_strides_dict[side] = {}

        for header in concatenated_stride_data:
            all_strides_array = np.concatenate(concatenated_stride_data[header][side])
            mean_strides_dict[side][header] = np.mean(all_strides_array, axis=0)
            std_strides_dict[side][header] = np.std(all_strides_array, axis=0)

    # Plot mean with standard deviation shaded region for each header and side
    plot_handles = []
    for side in ['Right', 'Left']:
        for header in mean_strides_dict[side]:
            plt.figure(figsize=(12, 6))
            plot_handle, = plt.plot(mean_strides_dict[side][header], label=f'{side} Side')
            plt.fill_between(np.arange(len(mean_strides_dict[side][header])),
                             mean_strides_dict[side][header] - std_strides_dict[side][header],
                             mean_strides_dict[side][header] + std_strides_dict[side][header],
                             alpha=0.3)
            plt.title(f'{header} ({side} Side) for {trial}')
            plt.xlabel('% Gait Cycle')
            plt.ylabel(f'{header} (deg)')
            #plt.legend()
            plot_handles.append(plot_handle)
            plt.show()
            plt.close()

    return plot_handles


In [ ]:
# Calculate length of each stride and then their mean and standard deviation
def clean_strides_mean_std(stride_list):
    lengths = [len(stride) for stride in stride_list]
    mean_length = np.mean(lengths)
    std_length = np.std(lengths)
    
    # Calculate the minimum acceptable length
    min_length = mean_length - std_length
    # Filter out strides that are shorter than the minimum acceptable length
    cleaned_strides = [stride for stride in stride_list if len(stride) >= min_length]
    
    return cleaned_strides


In [ ]:
# Calculate the interquartile range
def clean_strides_iqr(stride_list):
    lengths = [len(stride) for stride in stride_list]
    
    # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = np.percentile(lengths, 25)
    Q3 = np.percentile(lengths, 75)
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1
    
    # Calculate the lower bound for acceptable lengths
    lower_bound = Q1 - 1.5 * IQR
    # Filter out strides that are shorter than the lower bound
    cleaned_strides = [stride for stride in stride_list if len(stride) >= lower_bound]
    
    return cleaned_strides


In [ ]:
# Save the data to json files
def save_data_to_json(data, filepath):
    json_data = {header: {'Right': [arr.tolist() for arr in data[header]['Right']],
                          'Left': [arr.tolist() for arr in data[header]['Left']]}
                 for header in data}

    with open(filepath, 'w') as file:
        json.dump(json_data, file)


In [ ]:
# Function to detect if the stride data is upside down and flip it
def correct_orientation(data_array):
    if np.mean(data_array) < 0:  # Assuming that correct data has positive mean
        return -data_array
    return data_array


In [ ]:
#def process_side_strides(heels, header, data):
 #   strideList = []
  #  for idx in range(len(heels) - 1):
   #     start_index = int(heels[idx][0]) if hasattr(heels[idx], '__iter__') else int(heels[idx])
    #    end_index = int(heels[idx + 1][0]) if hasattr(heels[idx + 1], '__iter__') else int(heels[idx + 1])
        
        # Ensure indices are valid
     #   if start_index < end_index:
      #      stride_data = data.iloc[start_index:end_index][header].values
       #     stride_data = correct_orientation(stride_data)  # Correct orientation if necessary
        #    strideList.append(stride_data)
            
    #return strideList


In [ ]:
def segment_stride_phases(heel_indices, toe_indices, contralateral_heel_indices, contralateral_toe_indices, header, data, side, target_length=100):
    phase_normalized_strides = []
    phase_labels = []
    num_strides = len(heel_indices) - 1

    for idx in range(num_strides):
        start_idx = int(heel_indices[idx])
        end_idx = int(heel_indices[idx + 1])
        print(f"\nProcessing stride {idx + 1}/{num_strides}")#########
        if start_idx >= end_idx:
            continue
        print(f"Start index: {start_idx}, End index: {end_idx}")#####
        stride_data = data.iloc[start_idx:end_idx][header].values
        stride_length = len(stride_data)

        if stride_length < 2:
            print("Invalid stride (start_idx >= end_idx). Skipping.")####
            continue

        # Find contralateral toe-off candidates
        contralateral_toe_off_candidates = [toe for toe in contralateral_toe_indices if start_idx < toe < end_idx]
        if not contralateral_toe_off_candidates:
            continue
        contralateral_toe_off = int(min(contralateral_toe_off_candidates, key=lambda x: abs(x - start_idx)))

        # Find contralateral heel contact candidates
        contralateral_heel_candidates = [heel for heel in contralateral_heel_indices if start_idx < heel < end_idx]
        if not contralateral_heel_candidates:
            continue
        contralateral_heel_contact = int(min(contralateral_heel_candidates, key=lambda x: abs(x - contralateral_toe_off)))
       
        # Find ipsilateral toe-off as the toe index closest to start_idx
        ipsilateral_toe_off_candidates = [toe for toe in toe_indices if start_idx < toe < end_idx]
        if not ipsilateral_toe_off_candidates:
            continue
        ipsilateral_toe_off = int(min(ipsilateral_toe_off_candidates, key=lambda x: abs(x - start_idx)))

        # Segment the phases
        lr_segment = stride_data[0 : contralateral_toe_off - start_idx]
        mtst_segment = stride_data[contralateral_toe_off - start_idx : contralateral_heel_contact - start_idx]
        ps_segment = stride_data[contralateral_heel_contact - start_idx : ipsilateral_toe_off - start_idx]

        # Check for empty segments before interpolation
        if len(lr_segment) == 0 or len(mtst_segment) == 0 or len(ps_segment) == 0:
            continue

        # Normalize each phase individually
        lr_normalized = normalize_segment(lr_segment, 12)
        mtst_normalized = normalize_segment(mtst_segment, 38)
        ps_normalized = normalize_segment(ps_segment, 12)

        # Find the ipsilateral heel contact of the next stride for the remaining segment
        if idx + 1 < num_strides:
            next_start_idx = int(heel_indices[idx + 1])
            remaining_segment = data.iloc[ipsilateral_toe_off:next_start_idx][header].values
        else:
            remaining_segment = []

        # Normalize remaining segment to 38 points
        if len(remaining_segment) > 0:
            remaining_normalized = normalize_segment(remaining_segment, 38)
            final_normalized_stride = np.concatenate([lr_normalized, mtst_normalized, ps_normalized, remaining_normalized])
        else:
            final_normalized_stride = np.concatenate([lr_normalized, mtst_normalized, ps_normalized])

        # Ensure the final length is 100 points
        if len(final_normalized_stride) < target_length:
            remaining_length = target_length - len(final_normalized_stride)
            final_normalized_stride = np.concatenate([final_normalized_stride, np.zeros(remaining_length)])
        elif len(final_normalized_stride) > target_length:
            final_normalized_stride = final_normalized_stride[:target_length]

        phase_normalized_strides.append(final_normalized_stride)
        phase_labels.append((12, 38, 12, len(remaining_segment)))

    return phase_normalized_strides, phase_labels


In [ ]:
def normalize_segment(segment, target_length):
    if len(segment) < 1:
        return np.zeros(target_length)
    
    x_new = np.linspace(0, 1, target_length)
    x_old = np.linspace(0, 1, len(segment))
    
    if len(segment) < 4:
        interp_func = interp1d(x_old, segment, kind='linear', bounds_error=False, fill_value="extrapolate")
    else:
        interp_func = interp1d(x_old, segment, kind='cubic', bounds_error=False, fill_value="extrapolate")
    
    return interp_func(x_new)


In [ ]:
def plot_normalized_strides(phase_normalized_strides_r, phase_normalized_strides_l, header, phase_labels_r, phase_labels_l):
    # Determine the min and max values across both Right and Left side strides
    min_r = min([min(stride) for stride in phase_normalized_strides_r])
    max_r = max([max(stride) for stride in phase_normalized_strides_r])
    min_l = min([min(stride) for stride in phase_normalized_strides_l])
    max_l = max([max(stride) for stride in phase_normalized_strides_l])

    # Determine the common min and max values for y-axis
    common_min = (min(min_r, min_l) - 2)
    common_max = (max(max_r, max_l) + 2)

    # Create the plot
    plt.figure(figsize=(12, 6))

    # Plot Right side
    plt.subplot(1, 2, 1)
    for stride in phase_normalized_strides_r:
        if len(stride) == 100:  # Ensure that the stride is fully normalized
            plt.plot(stride, alpha=0.7)
        else:
            print(f"Stride length for right side is not 100: {len(stride)}")
    plt.title(f'{header} - Right Side')
    plt.xlabel('Normalized Points')
    plt.ylabel(header)
    plt.axvline(x=12, color='r', linestyle='--', label='LR end')
    plt.axvline(x=50, color='r', linestyle='--', label='MTSt end')
    plt.axvline(x=62, color='r', linestyle='--', label='PS end')

    # Set the same y-axis limits for both subplots
    plt.ylim([common_min, common_max])

    # Plot Left side
    plt.subplot(1, 2, 2)
    for stride in phase_normalized_strides_l:
        if len(stride) == 100:  # Ensure that the stride is fully normalized
            plt.plot(stride, alpha=0.7)
        else:
            print(f"Stride length for left side is not 100: {len(stride)}")
    plt.title(f'{header} - Left Side')
    plt.xlabel('Normalized Points')
    plt.ylabel(header)
    plt.axvline(x=12, color='r', linestyle='--', label='LR end')
    plt.axvline(x=50, color='r', linestyle='--', label='MTSt end')
    plt.axvline(x=62, color='r', linestyle='--', label='PS end')

    # Set the same y-axis limits for both subplots
    plt.ylim([common_min, common_max])

    plt.tight_layout()
    plt.show()


In [ ]:
def get_ROI (file, trial_name):
    print ('ROI = ', file['NexusData'][trial_name]['Info']['Region'][:])
    return file['NexusData'][trial_name]['Info']['Region'][:]

In [ ]:
def get_sampling_rate(file, trial_name):
    import numpy as np
    trial_data = file['NexusData'][trial_name]

    # Check if 'HeelType' exists in trial_data
    if 'HeelType' in trial_data:
        heel_type_data = trial_data['HeelType'][()]
        if isinstance(heel_type_data, bytes):
            heel_type = heel_type_data.decode('utf-8')
        elif isinstance(heel_type_data, np.ndarray):
            heel_type = ''.join(chr(code) for code in heel_type_data.flatten())
        else:
            heel_type = str(heel_type_data)
        print(f"HeelType: {heel_type}")

        if heel_type == 'IMUs':
            devices = trial_data['devices']
            # Find the first IMU device starting with 'TS0'
            imu_devices = [name for name in devices if name.startswith('TS0')]
            if not imu_devices:
                print('No IMU devices found starting with TS0. Using default sampling rate of 225 Hz.')
                return 225  # Default value
            imu_device = imu_devices[0]
            imu_device_data = devices[imu_device]

            if 'accel_x' in imu_device_data and 'Rate' in imu_device_data['accel_x']:
                sampling_rate = imu_device_data['accel_x']['Rate'][()]
                sampling_rate = np.array(sampling_rate).item()
                print('IMU heels Sampling =', sampling_rate)
                return sampling_rate
            else:
                print(f"Could not find 'Rate' under 'accel_x' in IMU device '{imu_device}'. Using default sampling rate of 225 Hz.")
                return 225  # Default value

        elif heel_type == 'Kinematics':
            print('Kinematic Heels = 100 Hz conversion')
            return 100
        else:
            print('Unknown HeelType:', heel_type)
            print('No definition of heel type; defaulting to 225 Hz conversion')
            return 225
    else:
        # 'HeelType' does not exist in trial_data
        print("HeelType does not exist in trial_data. Using default sampling rate of 225 Hz.")
        return 225  # Default sampling rate


In [ ]:
def indices_to_time(indices, sampling_rate, frame_offset=0):
    """
    Convert a list of indices to times in seconds based on the sampling rate.
    
    Parameters:
    - indices: list or array of indices (frame numbers).
    - sampling_rate: sampling rate in Hz.
    - frame_offset: frame offset to adjust indices if needed (default is 0).
    
    Returns:
    - times: list of times in seconds corresponding to the indices.
    """
    print('Indices = ', indices)
    indices = np.array(indices).flatten()
    indices = indices.tolist()
    times = [(idx - frame_offset) / sampling_rate for idx in indices]
    return times

In [ ]:
def filter_events_within_roi(events, ROI, event_sampling_rate):
    """
    Filters event indices to include only those within the ROI, handling the conversion
    between the ROI sampling rate (100 Hz) and the event sampling rate.
    """
    # Extract start and end frames from ROI
    roi_start_frame_100hz = ROI[0][0]
    roi_end_frame_100hz = ROI[1][0]
    
    # Convert ROI frames to time (seconds) using 100 Hz sampling rate
    roi_start_time = roi_start_frame_100hz / 100.0
    roi_end_time = roi_end_frame_100hz / 100.0
    #print(f"Start and end times: {roi_start_time} {roi_end_time}")
    
    # Ensure events is a 1D NumPy array of scalars
    events = np.array(events).flatten()
    
    # Convert event indices to time (seconds) using event_sampling_rate
    event_times = events / event_sampling_rate
    
    # Filter event times within ROI times using a boolean mask
    mask = (event_times >= roi_start_time) & (event_times <= roi_end_time)
    filtered_event_times = event_times[mask]
    
    # Convert filtered event times back to indices at event_sampling_rate
    filtered_events = np.round(filtered_event_times * event_sampling_rate).astype(int)
    
    # Convert to list
    filtered_events = np.array(filtered_events).flatten().tolist()
    
    return filtered_events

In [ ]:
def process_side_strides(ipsi_heels, ipsi_toes, contra_heels, contra_toes, header, data):
    """
    Processes strides for one side and collects associated events.

    Parameters:
    - ipsi_heels: list of ipsilateral heel strike times (seconds).
    - ipsi_toes: list of ipsilateral toe-off times (seconds).
    - contra_heels: list of contralateral heel strike times (seconds).
    - contra_toes: list of contralateral toe-off times (seconds).
    - header: string, the column name in data to process.
    - data: pandas DataFrame containing the data with a 'Time' column.

    Returns:
    - strideList: list of dictionaries, each containing stride data and events.
    """
    # Function to get indices from times
    def get_indices_from_times(event_times, data_times):
        indices = []
        for t in event_times:
            idx = np.argmin(np.abs(data_times - t))
            indices.append(idx)
            # Print event time, index, and corresponding data time
            #print(f"Event Time: {t:.6f}, Index: {idx}, Data Time at Index: {data_times[idx]:.6f}")
        return indices


    strideList = []

    # Extract data times
    data_times = data['time'].values

    # Convert event times to indices
    ipsi_heel_indices = get_indices_from_times(ipsi_heels, data_times)
    ipsi_toe_indices = get_indices_from_times(ipsi_toes, data_times)
    contra_heel_indices = get_indices_from_times(contra_heels, data_times)
    contra_toe_indices = get_indices_from_times(contra_toes, data_times)

    for idx in range(len(ipsi_heel_indices) - 1):
        start_idx = ipsi_heel_indices[idx]
        end_idx = ipsi_heel_indices[idx + 1]
        start_time = data_times[start_idx]
        end_time = data_times[end_idx]
        #print(f'\nStride {idx + 1}: Start Time = {start_time:.3f}s, End Time = {end_time:.3f}s')

        # Extract stride data between start_idx and end_idx
        stride_data = data.iloc[start_idx:end_idx][header].values
        stride_time = data.iloc[start_idx:end_idx]['time'].values

        # Print stride data length and a sample
        #print(f'Stride Data Length: {len(stride_data)}')
        if len(stride_data) > 0:
            pass
        else:
            print('No stride data found for this stride.')
            continue  # Skip to the next stride if there's no data

        events_in_stride = []
        all_events = []
        for t in ipsi_toes:
            if start_time <= t <= end_time:
                all_events.append({'time': t, 'type': 'ipsi_toe_off'})
        # Contralateral events
        for t in contra_toes:
            if start_time <= t <= end_time:
                all_events.append({'time': t, 'type': 'contra_toe_off'})
        for t in contra_heels:
            if start_time <= t <= end_time:
                all_events.append({'time': t, 'type': 'contra_heel_strike'})

        # Sort events by time to maintain sequence
        all_events.sort(key=lambda x: x['time'])

        # Calculate event indices relative to stride_data
        for event in all_events:
            relative_time = (event['time'] - start_time) / (end_time - start_time)
            event_idx = int(relative_time * (len(stride_data) - 1))
            event['index'] = event_idx
            events_in_stride.append(event)

        # Correct orientation if necessary
        stride_data = correct_orientation(stride_data)

        # Store stride data and events
        stride_info = {
            'stride_data': stride_data,
            'start_time': start_time,
            'end_time': end_time,
            'events': events_in_stride
        }

        strideList.append(stride_info)

    return strideList

In [ ]:
def plot_strides(strides_r, strides_l, header, header_type, plot_style='single', use_normalized_data=True):
    """
    Plots the strides for the given header and header type, including gait events.

    Parameters:
    - strides_r: list of stride_info dictionaries for the right side (or None)
    - strides_l: list of stride_info dictionaries for the left side (or None)
    - header: the header name
    - header_type: 'bilateral', 'right_only', 'left_only'
    - plot_style: 'single' or 'side_by_side'
    - use_normalized_data: Boolean indicating whether to use normalized data
    """
    import numpy as np
    import matplotlib.pyplot as plt

    # Define event markers and colors
    event_markers = {
        'ipsi_toe_off': {'marker': 'o', 'color': 'green', 'label': 'Ipsi Toe Off'},
        'contra_toe_off': {'marker': 'x', 'color': 'purple', 'label': 'Contra Toe Off'},
        'contra_heel_strike': {'marker': '^', 'color': 'orange', 'label': 'Contra Heel Strike'},
    }

    def plot_events(stride_info, stride_data, x_values, use_normalized_data):
        """
        Plots the gait events on the stride data.

        Parameters:
        - stride_info: Dictionary containing stride information.
        - stride_data: The stride data to plot.
        - x_values: The x-axis values corresponding to stride_data.
        - use_normalized_data: Boolean indicating whether to use normalized event indices.
        """
        # Select events and indices based on whether normalized data is used
        if use_normalized_data:
            events = stride_info.get('normalized_events', stride_info['events'])
            for event in events:
                idx = event.get('normalized_index')
                event_type = event['type']
                marker_info = event_markers.get(event_type, {'marker': 'd', 'color': 'black', 'label': event_type})
                if idx is not None and 0 <= idx < len(x_values):
                    plt.plot(x_values[idx], stride_data[idx], marker=marker_info['marker'], color=marker_info['color'],
                             markersize=8, label=marker_info['label'])
        else:
            events = stride_info['events']
            for event in events:
                idx = event['index']
                event_type = event['type']
                marker_info = event_markers.get(event_type, {'marker': 'd', 'color': 'black', 'label': event_type})
                if idx is not None and 0 <= idx < len(x_values):
                    plt.plot(x_values[idx], stride_data[idx], marker=marker_info['marker'], color=marker_info['color'],
                             markersize=8, label=marker_info['label'])

    # Begin plotting
    if header_type == 'bilateral':
        if plot_style == 'single':
            plt.figure()
            # Plot right strides in red
            for stride_info in strides_r:
                if use_normalized_data and 'normalized_stride_data' in stride_info:
                    stride_data = stride_info['normalized_stride_data']
                    x_values = np.linspace(0, 100, len(stride_data))  # Stride percentage
                else:
                    stride_data = stride_info['stride_data']
                    x_values = np.arange(len(stride_data))
                plt.plot(x_values, stride_data, color='red', alpha=0.5)
                plot_events(stride_info, stride_data, x_values, use_normalized_data)

            # Plot left strides in blue
            for stride_info in strides_l:
                if use_normalized_data and 'normalized_stride_data' in stride_info:
                    stride_data = stride_info['normalized_stride_data']
                    x_values = np.linspace(0, 100, len(stride_data))
                else:
                    stride_data = stride_info['stride_data']
                    x_values = np.arange(len(stride_data))
                plt.plot(x_values, stride_data, color='blue', alpha=0.5)
                plot_events(stride_info, stride_data, x_values, use_normalized_data)

            plt.title(f'Bilateral Strides - {header}')
            plt.xlabel('Stride Percentage (%)' if use_normalized_data else 'Sample Points')
            plt.ylabel(header)
            # Remove duplicate legend entries
            handles, labels = plt.gca().get_legend_handles_labels()
            by_label = dict(zip(labels, handles))
            plt.legend(by_label.values(), by_label.keys())
            plt.show()

        elif plot_style == 'side_by_side':
            fig, axs = plt.subplots(1, 2, figsize=(12, 6))
            # Plot right strides
            for stride_info in strides_r:
                if use_normalized_data and 'normalized_stride_data' in stride_info:
                    stride_data = stride_info['normalized_stride_data']
                    x_values = np.linspace(0, 100, len(stride_data))
                else:
                    stride_data = stride_info['stride_data']
                    x_values = np.arange(len(stride_data))
                axs[0].plot(x_values, stride_data, color='red', alpha=0.5)
                # Plot events
                plot_events(stride_info, stride_data, x_values, use_normalized_data)

            axs[0].set_title(f'Right Strides - {header}')
            axs[0].set_xlabel('Stride Percentage (%)' if use_normalized_data else 'Sample Points')
            axs[0].set_ylabel(header)

            # Plot left strides
            for stride_info in strides_l:
                if use_normalized_data and 'normalized_stride_data' in stride_info:
                    stride_data = stride_info['normalized_stride_data']
                    x_values = np.linspace(0, 100, len(stride_data))
                else:
                    stride_data = stride_info['stride_data']
                    x_values = np.arange(len(stride_data))
                axs[1].plot(x_values, stride_data, color='blue', alpha=0.5)
                # Plot events
                plot_events(stride_info, stride_data, x_values, use_normalized_data)

            axs[1].set_title(f'Left Strides - {header}')
            axs[1].set_xlabel('Stride Percentage (%)' if use_normalized_data else 'Sample Points')
            axs[1].set_ylabel(header)

            # Adjust layout and show plot
            plt.tight_layout()
            # Remove duplicate legend entries
            handles, labels = axs[0].get_legend_handles_labels()
            by_label = dict(zip(labels, handles))
            axs[0].legend(by_label.values(), by_label.keys())
            plt.show()

    elif header_type == 'right_only':
        plt.figure()
        for stride_info in strides_r:
            if use_normalized_data and 'normalized_stride_data' in stride_info:
                stride_data = stride_info['normalized_stride_data']
                x_values = np.linspace(0, 100, len(stride_data))
            else:
                stride_data = stride_info['stride_data']
                x_values = np.arange(len(stride_data))
            plt.plot(x_values, stride_data, color='red', alpha=0.5)
            plot_events(stride_info, stride_data, x_values, use_normalized_data)

        plt.title(f'Right Strides - {header}')
        plt.xlabel('Stride Percentage (%)' if use_normalized_data else 'Sample Points')
        plt.ylabel(header)
        # Remove duplicate legend entries
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys())
        plt.show()

    elif header_type == 'left_only':
        plt.figure()
        for stride_info in strides_l:
            if use_normalized_data and 'normalized_stride_data' in stride_info:
                stride_data = stride_info['normalized_stride_data']
                x_values = np.linspace(0, 100, len(stride_data))
            else:
                stride_data = stride_info['stride_data']
                x_values = np.arange(len(stride_data))
            plt.plot(x_values, stride_data, color='blue', alpha=0.5)
            plot_events(stride_info, stride_data, x_values, use_normalized_data)

        plt.title(f'Left Strides - {header}')
        plt.xlabel('Stride Percentage (%)' if use_normalized_data else 'Sample Points')
        plt.ylabel(header)
        # Remove duplicate legend entries
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys())
        plt.show()


In [ ]:
def normalize_strides(strides_list, normalization_type='cycle', target_length=100):
    """
    Normalizes strides based on the specified normalization type.

    Parameters:
    - strides_list: List of stride_info dictionaries.
    - normalization_type: 'cycle' or 'phase'.
    - target_length: Integer, the target length for normalization.

    Returns:
    - normalized_strides_list: List of stride_info dictionaries with normalized stride_data and adjusted events.
    """
    normalized_strides_list = []

    for stride_info in strides_list:
        stride_data = stride_info['stride_data']
        stride_length = len(stride_data)

        if normalization_type == 'cycle':
            # Normalize stride data to target_length using interpolation
            stride_data_normalized = np.interp(
                np.linspace(0, stride_length - 1, target_length),
                np.arange(stride_length),
                stride_data
            )

            # Adjust event indices
            adjusted_events = []
            for event in stride_info['events']:
                original_index = event['index']
                normalized_index = int((original_index / (stride_length - 1)) * (target_length - 1))
                event_adjusted = event.copy()
                event_adjusted['index'] = normalized_index
                adjusted_events.append(event_adjusted)

            # Update stride_info with normalized data and adjusted events
            stride_info_normalized = stride_info.copy()
            stride_info_normalized['normalized_stride_data'] = stride_data_normalized
            stride_info_normalized['normalized_events'] = adjusted_events

        elif normalization_type == 'phase':
            # Placeholder for 'phase' normalization (to be developed)
            stride_info_normalized = stride_info  # No changes for now

        else:
            raise ValueError("Invalid normalization_type. Choose 'cycle' or 'phase'.")

        normalized_strides_list.append(stride_info_normalized)

    return normalized_strides_list


In [ ]:
def apply_conventions(strides_r, strides_l, header):
    """
    Applies the required processing to the strides for the given header.

    Parameters:
    - strides_r: List of stride_info dictionaries for the right side (or None).
    - strides_l: List of stride_info dictionaries for the left side (or None).
    - header: The header name (string).

    Returns:
    - strides_r: Processed strides for the right side.
    - strides_l: Processed strides for the left side.
    """
    import numpy as np

    # Headers to skip
    skip_headers = [
        'time', 'pelvis_tilt', 'pelvis_list', 'pelvis_tx', 'pelvis_ty', 'pelvis_tz',
        'hip_rotation_r', 'hip_rotation_l', 'subtalar_angle_l', 'subtalar_angle_r',
        'mtp_angle_l', 'mtp_angle_r'
    ]

    if header in skip_headers:
        # Copy normalized data to processed data without changes
        if strides_r is not None:
            for stride_info in strides_r:
                stride_info['processed_stride_data'] = stride_info['normalized_stride_data']
        if strides_l is not None:
            for stride_info in strides_l:
                stride_info['processed_stride_data'] = stride_info['normalized_stride_data']
        return strides_r, strides_l

    # Headers where right strides need to be multiplied by -1
    headers_flip_right = ['pelvis_rotation', 'lumbar_bending', 'lumbar_rotation']

    # Headers where both sides need to be multiplied by -1
    headers_multiply_minus1 = ['lumbar_extension']

    # Headers where we need to check for flipped strides
    headers_check_flip = ['hip_flexion', 'knee_angle', 'ankle_angle']

    # Process right strides
    if strides_r is not None:
        # Collect all normalized stride data for right side
        all_stride_data_r = [stride_info['normalized_stride_data'] for stride_info in strides_r]

        for stride_info in strides_r:
            # Start with a copy of the normalized data
            stride_data = stride_info['normalized_stride_data'].copy()

            # Apply conventions
            if header in headers_flip_right:
                stride_data = -stride_data

            if header in headers_multiply_minus1:
                stride_data = -stride_data

            if any(h in header for h in headers_check_flip) or header == 'hip_adduction_r':
                # Correct flipped strides using correlation with mean stride
                stride_data = correct_flipped_stride(stride_data, all_stride_data_r)

            # Demean if necessary
            if header == 'pelvis_rotation':
                mean_value = np.mean(stride_data)
                stride_data -= mean_value

            # Store the processed data
            stride_info['processed_stride_data'] = stride_data

    # Process left strides similarly
    if strides_l is not None:
        # Collect all normalized stride data for left side
        all_stride_data_l = [stride_info['normalized_stride_data'] for stride_info in strides_l]

        for stride_info in strides_l:
            # Start with a copy of the normalized data
            stride_data = stride_info['normalized_stride_data'].copy()

            # Apply conventions
            if header in headers_multiply_minus1:
                stride_data = -stride_data

            if any(h in header for h in headers_check_flip) or header == 'hip_adduction_l':
                # Correct flipped strides using correlation with mean stride
                stride_data = correct_flipped_stride(stride_data, all_stride_data_l)

            # Demean if necessary
            if header == 'pelvis_rotation':
                mean_value = np.mean(stride_data)
                stride_data -= mean_value

            # Store the processed data
            stride_info['processed_stride_data'] = stride_data

    return strides_r, strides_l



def correct_flipped_stride(stride_data, all_stride_data_list):
    """
    Identifies and corrects a single stride if it is flipped, based on correlation with the mean stride.

    Parameters:
    - stride_data: Numpy array of stride data.
    - all_stride_data_list: List of numpy arrays containing all stride data.

    Returns:
    - corrected_stride_data: Corrected stride data.
    """

    # Compute the mean stride from all strides
    all_strides_array = np.stack(all_stride_data_list, axis=0)
    mean_stride = np.mean(all_strides_array, axis=0)

    # Compute the correlation coefficient between the stride and the mean stride
    correlation = np.corrcoef(stride_data, mean_stride)[0, 1]

    # If the correlation is negative, flip the stride
    if correlation < 0:
        corrected_stride_data = -stride_data
    else:
        corrected_stride_data = stride_data

    return corrected_stride_data


def correct_flipped_stride_hip_adduction(stride_data):
    """
    Identifies and corrects a single hip adduction stride if it is flipped.

    Parameters:
    - stride_data: Numpy array of stride data.

    Returns:
    - stride_data: Corrected stride data.
    """
    # Analyze values at specific points
    idx_20 = int(0.2 * len(stride_data))
    idx_80 = int(0.8 * len(stride_data))
    value_20 = stride_data[idx_20]
    value_80 = stride_data[idx_80]
    # If the pattern is inverted, flip the stride
    if value_20 < value_80:
        stride_data = -stride_data
    return stride_data

In [ ]:
def plot_processed_strides(strides_r, strides_l, header, header_type, plot_style='single'):
    """
    Plots the processed strides for the given header and header type.

    Parameters:
    - strides_r: list of stride_info dictionaries for the right side (or None)
    - strides_l: list of stride_info dictionaries for the left side (or None)
    - header: the header name
    - header_type: 'bilateral', 'right_only', 'left_only'
    - plot_style: 'single' or 'side_by_side' (optional, default is 'single')
    """
    import numpy as np
    import matplotlib.pyplot as plt

    # Begin plotting
    if header_type == 'bilateral':
        if plot_style == 'single':
            plt.figure()
            # Plot right processed strides in red
            if strides_r is not None:
                for stride_info in strides_r:
                    stride_data = stride_info['processed_stride_data']
                    x_values = np.linspace(0, 100, len(stride_data))  # Stride percentage
                    plt.plot(x_values, stride_data, color='red', alpha=0.5)
            # Plot left processed strides in blue
            if strides_l is not None:
                for stride_info in strides_l:
                    stride_data = stride_info['processed_stride_data']
                    x_values = np.linspace(0, 100, len(stride_data))
                    plt.plot(x_values, stride_data, color='blue', alpha=0.5)
            plt.title(f'Processed Bilateral Strides - {header}')
            plt.xlabel('Stride Percentage (%)')
            plt.ylabel(header)
            plt.legend(['Right Processed Strides', 'Left Processed Strides'])
            plt.show()
        elif plot_style == 'side_by_side':
            fig, axs = plt.subplots(1, 2, figsize=(12, 6))
            # Plot right processed strides
            if strides_r is not None:
                for stride_info in strides_r:
                    stride_data = stride_info['processed_stride_data']
                    x_values = np.linspace(0, 100, len(stride_data))
                    axs[0].plot(x_values, stride_data, color='red', alpha=0.5)
                axs[0].set_title(f'Processed Right Strides - {header}')
                axs[0].set_xlabel('Stride Percentage (%)')
                axs[0].set_ylabel(header)
            # Plot left processed strides
            if strides_l is not None:
                for stride_info in strides_l:
                    stride_data = stride_info['processed_stride_data']
                    x_values = np.linspace(0, 100, len(stride_data))
                    axs[1].plot(x_values, stride_data, color='blue', alpha=0.5)
                axs[1].set_title(f'Processed Left Strides - {header}')
                axs[1].set_xlabel('Stride Percentage (%)')
                axs[1].set_ylabel(header)
            plt.tight_layout()
            plt.show()
    elif header_type == 'right_only':
        plt.figure()
        if strides_r is not None:
            for stride_info in strides_r:
                stride_data = stride_info['processed_stride_data']
                x_values = np.linspace(0, 100, len(stride_data))
                plt.plot(x_values, stride_data, color='red', alpha=0.5)
        plt.title(f'Processed Right Strides - {header}')
        plt.xlabel('Stride Percentage (%)')
        plt.ylabel(header)
        plt.legend(['Right Processed Strides'])
        plt.show()
    elif header_type == 'left_only':
        plt.figure()
        if strides_l is not None:
            for stride_info in strides_l:
                stride_data = stride_info['processed_stride_data']
                x_values = np.linspace(0, 100, len(stride_data))
                plt.plot(x_values, stride_data, color='blue', alpha=0.5)
        plt.title(f'Processed Left Strides - {header}')
        plt.xlabel('Stride Percentage (%)')
        plt.ylabel(header)
        plt.legend(['Left Processed Strides'])
        plt.show()


In [ ]:
# Main script to process all trials
strideList_dict = {}
#for trial in range(1, 22): Mod for simplification. 
for trial in range(16,22): #Participants
    current_folderpath = Path(folderpath) / f"{trial:02d}"
    current_HeelPath = HeelPathTemplate.format(f"{trial:02d}")
    mot_files = list((current_folderpath / 'IKResults').glob("*.mot"))
    json_save_path = os.path.join(base_json_path, f"Subject{trial:02d}.json")
    participant_id = f"Participant_{trial:02d}"  
    print('Participant ID: ', participant_id)
    strideList_dict[participant_id] = {}
    
    
    for file_path in mot_files: #Participants trials
        print('Current File path: ', file_path)
        file_path_str = str(file_path)
        data = pd.read_csv(file_path, sep='\t', header=header_line)
        current_headers = data.columns
        
        # Extract trial name from file path
        match = re.search(pattern, file_path_str)
        if match:
            trial_name = match.group(1)
            print ('Processing Trial:', trial_name)
            print('HeelFilePath: ', current_HeelPath)
            # Check if trial exists in the HDF5 file
            with h5py.File(current_HeelPath, 'r') as file:
                print('HeelFilePath: ', current_HeelPath)
                if trial_name in file['NexusData']:
                    heels_r = []
                    heels_l = []
                    toes_r = []
                    toes_l = []
                    
                    try:
                        # Attempt to load heels and toes data
                        for side in ['Right', 'Left']:
                            try:
                                # Try accessing data in the original structure
                                if isinstance(file['NexusData'][trial_name][side]['heels'], h5py.Dataset):
                                    if side == 'Right':
                                        nexus_data_r = file['NexusData'][trial_name][side]['heels'][:]
                                    else:
                                        nexus_data_l = file['NexusData'][trial_name][side]['heels'][:]
                                    print(f'{side} Heels loaded in original structure.')
                                else:
                                    # If it’s a group, try accessing the inner dataset
                                    if side == 'Right':
                                        nexus_data_r = file['NexusData'][trial_name][side]['heels']['heels'][:]
                                    else:
                                        nexus_data_l = file['NexusData'][trial_name][side]['heels']['heels'][:]
                                    print(f'{side} Heels loaded in alternative structure: ["heels"]["heels"].')
        
                            except KeyError:
                                print(f'Error: {side} Heels not found for trial {trial_name}')
        
                            try:
                                # Try accessing data in the original structure for Toes
                                if isinstance(file['NexusData'][trial_name][side]['Toes'], h5py.Dataset):
                                    if side == 'Right':
                                        nexus_toes_r = file['NexusData'][trial_name][side]['Toes'][:]
                                    else:
                                        nexus_toes_l = file['NexusData'][trial_name][side]['Toes'][:]
                                    print(f'{side} Toes loaded in original structure.')
                                else:
                                    # If it’s a group, try accessing the inner dataset
                                    if side == 'Right':
                                        nexus_toes_r = file['NexusData'][trial_name][side]['Toes']['Toes'][:]
                                    else:
                                        nexus_toes_l = file['NexusData'][trial_name][side]['Toes']['Toes'][:]
                                    print(f'{side} Toes loaded in alternative structure: ["Toes"]["Toes"].')
        
                            except KeyError:
                                print(f'Error: {side} Toes not found for trial {trial_name}')
                                
                    except KeyError: #this one
                            print(f'Error: {side} Toes not found for trial {trial_name}')



                    
                    # Process Heel - Toe data
                    ROI = get_ROI(file,trial_name)
                    sampling = get_sampling_rate(file,trial_name)
                    print('sampling rate: ', sampling)
                    #get rid of events outside ROI
                    # Filter events
                    heels_r_filtered = filter_events_within_roi(nexus_data_r, ROI, sampling)
                    heels_l_filtered = filter_events_within_roi(nexus_data_l, ROI, sampling)
                    toes_r_filtered = filter_events_within_roi(nexus_toes_r, ROI, sampling)
                    toes_l_filtered = filter_events_within_roi(nexus_toes_l, ROI, sampling)
                    print('Heels Filtered', heels_r_filtered)

                    heels_r_times = indices_to_time(heels_r_filtered, sampling_rate=sampling)
                    heels_l_times = indices_to_time(heels_l_filtered, sampling_rate=sampling)
                    toes_r_times = indices_to_time(toes_r_filtered, sampling_rate=sampling)
                    toes_l_times = indices_to_time(toes_l_filtered, sampling_rate=sampling)                    
                    #####################################
                    heels_r_sorted = sorted(heels_r_times)
                    heels_l_sorted = sorted(heels_l_times)
                    toes_r_sorted = sorted(toes_r_times)
                    toes_l_sorted = sorted(toes_l_times)
                    
                    # Combine all event times for x-axis limits
                    all_event_times = heels_r_sorted + heels_l_sorted + toes_r_sorted + toes_l_sorted
                    
                    if all_event_times:
                        min_time = min(all_event_times)
                        max_time = max(all_event_times)
                    else:
                        print("No events to plot.")
                        min_time = 0
                        max_time = 1  # Set default values or handle as needed
                    
                    plt.figure(figsize=(15, 5))
                    # Plot Right Heel Contacts
                    if heels_r_sorted:
                        plt.scatter(heels_r_sorted, [1]*len(heels_r_sorted), marker='^', color='blue', label='Right Heel Contact')
                    # Plot Left Heel Contacts
                    if heels_l_sorted:
                        plt.scatter(heels_l_sorted, [1]*len(heels_l_sorted), marker='^', color='red', label='Left Heel Contact')
                    
                    # Plot Right Toe-Offs
                    if toes_r_sorted:
                        plt.scatter(toes_r_sorted, [0]*len(toes_r_sorted), marker='v', color='blue', label='Right Toe-Off')
                    
                    # Plot Left Toe-Offs
                    if toes_l_sorted:
                        plt.scatter(toes_l_sorted, [0]*len(toes_l_sorted), marker='v', color='red', label='Left Toe-Off')
                    
                    # Adjust y positions slightly for visualization
                    plt.yticks([0, 1], ['Toe-Off', 'Heel Contact'])
                    
                    # Add vertical lines for visualization
                    for t in heels_r_sorted:
                        plt.axvline(x=t, color='blue', linestyle='--', alpha=0.3)
                    for t in heels_l_sorted:
                        plt.axvline(x=t, color='red', linestyle='--', alpha=0.3)
                    for t in toes_r_sorted:
                        plt.axvline(x=t, color='blue', linestyle=':', alpha=0.3)
                    for t in toes_l_sorted:
                        plt.axvline(x=t, color='red', linestyle=':', alpha=0.3)
                    
                    # Customize the plot
                    plt.xlabel('Time (seconds)')
                    plt.ylabel('Event Type')
                    plt.title(f'Gait Events Timeline for Trial {trial_name}')
                    plt.legend(loc='upper right')
                    plt.xlim([min_time - 0.5, max_time + 0.5])
                    plt.tight_layout()
                    plt.show()


                    ##################################### Stride Segmentation and Normalisation
                    # Process strides for each header
                    bilateral_headers = []
                    right_only_headers = []
                    left_only_headers = []
                    trial_processed_strides = {}
                    for header in data.columns:
                        if header.endswith('_r'):
                            right_only_headers.append(header)
                        elif header.endswith('_l'):
                            left_only_headers.append(header)
                        else:
                            bilateral_headers.append(header)

                    for header in bilateral_headers:
                        print('Processing bilateral header:', header)
                        # Process for right strides
                        strides_r = process_side_strides(
                            ipsi_heels=heels_r_times,
                            ipsi_toes=toes_r_times,
                            contra_heels=heels_l_times,
                            contra_toes=toes_l_times,
                            header=header,
                            data=data
                        )
                        strides_r = normalize_strides(strides_r, normalization_type='cycle', target_length=100)
                        
                        # Process for left strides
                        strides_l = process_side_strides(
                            ipsi_heels=heels_l_times,
                            ipsi_toes=toes_l_times,
                            contra_heels=heels_r_times,
                            contra_toes=toes_r_times,
                            header=header,
                            data=data
                        )
                        strides_l = normalize_strides(strides_l, normalization_type='cycle', target_length=100)

                        strides_r, strides_l = apply_conventions(strides_r, strides_l, header)
                        
                        #plot_strides(strides_r, strides_l, header, 'bilateral', plot_style='single')

                        trial_processed_strides[header] = {
                            'strides_r': strides_r,
                            'strides_l': strides_l,
                            'header_type': 'bilateral'
                        }

                    
                    for header in right_only_headers:
                        print('Processing right-only header:', header)
                        # Process for right strides
                        strides_r = process_side_strides(
                            ipsi_heels=heels_r_times,
                            ipsi_toes=toes_r_times,
                            contra_heels=heels_l_times,
                            contra_toes=toes_l_times,
                            header=header,
                            data=data
                        )
                        strides_r = normalize_strides(strides_r, normalization_type='cycle', target_length=100)
                        strides_r, _ = apply_conventions(strides_r, None, header)
                        #plot_strides(strides_r, None, header, 'right_only')
                        trial_processed_strides[header] = {
                            'strides_r': strides_r,
                            'strides_l': None,
                            'header_type': 'right_only'
                        }

                    
                    for header in left_only_headers:
                        print('Processing left-only header:', header)
                        # Process for left strides
                        strides_l = process_side_strides(
                            ipsi_heels=heels_l_times,
                            ipsi_toes=toes_l_times,
                            contra_heels=heels_r_times,
                            contra_toes=toes_r_times,
                            header=header,
                            data=data
                        )
                        strides_l = normalize_strides(strides_l, normalization_type='cycle', target_length=100)
                        _, strides_l = apply_conventions(None, strides_l, header)
                        #plot_strides(None, strides_l, header, 'left_only')
                    
                        trial_processed_strides[header] = {
                            'strides_r': None,
                            'strides_l': strides_l,
                            'header_type': 'left_only'
                        }


                    for header, stride_data in trial_processed_strides.items():
                        strides_r = stride_data['strides_r']
                        strides_l = stride_data['strides_l']
                        header_type = stride_data['header_type']
                        plot_processed_strides(strides_r, strides_l, header, header_type, plot_style='single')
                    

                    
    
                    
                    # Store trial processed strides under participant and trial
                    strideList_dict[participant_id][trial_name] = trial_processed_strides
                    for header, stride_data in trial_processed_strides.items():
                        strides_r = stride_data['strides_r']
                        strides_l = stride_data['strides_l']
                        header_type = stride_data['header_type']
                    
                        # Initialize a new figure
                        plt.figure()
                    
                        # Plot right strides
                        if strides_r:
                            mean_stride_r, lower_r, upper_r = compute_mean_and_variability(strides_r, variability='CI')
                            plot_mean_and_variability(mean_stride_r, lower_r, upper_r, header, 'Right', 'red')
                    
                        # Plot left strides
                        if strides_l:
                            mean_stride_l, lower_l, upper_l = compute_mean_and_variability(strides_l, variability='CI')
                            plot_mean_and_variability(mean_stride_l, lower_l, upper_l, header, 'Left', 'blue')
                    
                        # Set the title and show the plot
                        plt.title(f'{header} - Mean and Variability')
                        plt.xlabel('Stride Percentage (%)')
                        plt.ylabel(header)
                        plt.legend()
                        plt.show()
            #raise Exception("Stopping script for testing.")
    save_participant_strides(strideList_dict, participant_id, current_folderpath)

In [ ]:
def compute_mean_and_variability(strides_list, variability='SD'):
    """
    Computes the mean stride and variability (SD, SE, or 95% CI) across a list of strides.

    Parameters:
    - strides_list: List of stride_info dictionaries containing 'processed_stride_data'.
    - variability: Type of variability to compute ('SD', 'SE', 'CI').

    Returns:
    - mean_stride: Numpy array of the mean stride.
    - lower_bound: Numpy array of the lower bound of the variability area.
    - upper_bound: Numpy array of the upper bound of the variability area.
    """
    import numpy as np

    if not strides_list:
        return None, None, None

    # Collect all processed stride data
    stride_data_list = [stride_info['processed_stride_data'] for stride_info in strides_list]
    # Stack stride data into a 2D array (strides x data points)
    stride_data_array = np.stack(stride_data_list, axis=0)
    # Compute the mean across strides
    mean_stride = np.mean(stride_data_array, axis=0)

    # Compute variability
    if variability == 'SD':
        var_stride = np.std(stride_data_array, axis=0)
        lower_bound = mean_stride - var_stride
        upper_bound = mean_stride + var_stride
    elif variability == 'SE':
        var_stride = np.std(stride_data_array, axis=0) / np.sqrt(stride_data_array.shape[0])
        lower_bound = mean_stride - var_stride
        upper_bound = mean_stride + var_stride
    elif variability == 'CI':
        from scipy import stats
        confidence = 0.95
        n = stride_data_array.shape[0]
        stderr = stats.sem(stride_data_array, axis=0)
        t_value = stats.t.ppf((1 + confidence) / 2., n - 1)
        margin_of_error = t_value * stderr
        lower_bound = mean_stride - margin_of_error
        upper_bound = mean_stride + margin_of_error
    else:
        raise ValueError("Invalid variability type. Choose 'SD', 'SE', or 'CI'.")

    return mean_stride, lower_bound, upper_bound


def plot_mean_and_variability(mean_stride, lower_bound, upper_bound, header, side_label, color):
    """
    Plots the mean stride and variability area.

    Parameters:
    - mean_stride: Numpy array of the mean stride.
    - lower_bound: Numpy array of the lower bound of the variability area.
    - upper_bound: Numpy array of the upper bound of the variability area.
    - header: The parameter name (string) being plotted.
    - side_label: Label for the side ('Right' or 'Left').
    - color: Color for the plot.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    if mean_stride is None:
        return

    x_values = np.linspace(0, 100, len(mean_stride))  # Stride percentage

    plt.plot(x_values, mean_stride, color=color, label=f'{side_label} Mean')
    plt.fill_between(x_values, lower_bound, upper_bound, color=color, alpha=0.3, label=f'{side_label} Variability')
    plt.xlabel('Stride Percentage (%)')
    plt.ylabel(header)
    plt.legend()

In [ ]:
def save_participant_strides(strideList_dict, participant_id, participant_folderpath):
    """
    Saves the participant's concatenated strides into the Kinematics folder.

    Parameters:
    - strideList_dict: Dictionary containing all participants' strides.
    - participant_id: String identifier for the participant.
    - participant_folderpath: Path object of the participant's folder.
    """
    import os
    import pickle

    # Access the participant's data from strideList_dict
    participant_data = strideList_dict[participant_id]

    # Initialize a dictionary to store concatenated strides across trials
    participant_concatenated_strides = {}

    # Iterate over trials and concatenate strides
    for trial_name, trial_processed_strides in participant_data.items():
        for header, stride_data in trial_processed_strides.items():
            # Initialize the header in participant_concatenated_strides if not already present
            if header not in participant_concatenated_strides:
                participant_concatenated_strides[header] = {
                    'strides_r': [],
                    'strides_l': [],
                    'header_type': stride_data['header_type']
                }

            # Extend the participant's strides with the strides from this trial
            if stride_data['strides_r']:
                participant_concatenated_strides[header]['strides_r'].extend(stride_data['strides_r'])
            if stride_data['strides_l']:
                participant_concatenated_strides[header]['strides_l'].extend(stride_data['strides_l'])

    # Define the path to the Kinematics folder
    kinematics_folder = participant_folderpath / 'Kinematics'

    # Create the Kinematics folder if it doesn't exist
    if not kinematics_folder.exists():
        os.makedirs(kinematics_folder)
        print(f"Created Kinematics folder at {kinematics_folder}")

    # Define the file path to save the strides
    strides_file_path = kinematics_folder / 'concatenated_strides.pkl'

    # Save the participant_concatenated_strides dictionary to the file
    with open(strides_file_path, 'wb') as f:
        pickle.dump(participant_concatenated_strides, f)

    print(f"Saved concatenated strides to {strides_file_path}")


In [ ]:
from scipy.io import loadmat

try:
    mat_data = loadmat(current_HeelPath)
    print("Successfully loaded data using `loadmat`.")
    # You can then explore the keys in the file to understand its structure
    print(mat_data.keys())
except NotImplementedError:
    print(f"Error: The file '{current_HeelPath}' may be saved in a newer HDF5-compatible format.")
except Exception as e:
    print(f"Failed to load '{current_HeelPath}': {e}")